In [1]:
## Load R libraries.

inLibraries = list('repr','rgdal','gstat','sp','spdep','rgeos','maptools','RColorBrewer','classInt','raster')
for (rpack in inLibraries) {
  if (is.element(rpack,installed.packages()[,1])){           
      #Load the library into R
      suppressMessages(library(rpack,character.only = TRUE))
    }
    else {
        print(paste("Warning:  ",rpack," is not an installed package"))
    }
}

library(geojsonio)

library(tidyverse)

library(sjstats)


options(repr.plot.width=15, repr.plot.height=15)

Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson


Attaching package: ‘geojsonio’


The following object is masked from ‘package:base’:

    pretty


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.0     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract() masks raster::extract()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ dplyr::select()  masks raster::select()


Attaching package: ‘sjstats’


The following object is masked from ‘package:raster’:

    cv




In [2]:
all_falls <- read.csv('data/results/all_falls.csv')

all <- read.csv('data/results/all.csv')




In [3]:
# Creating Train and Test datasets

# all_falls

set.seed(100)
sample_size <- floor(0.7 * nrow(all_falls))

train_ind <- sample(seq_len(nrow(all_falls)), size = sample_size)

all_falls_train <- all_falls[train_ind, ]
all_falls_test <- all_falls[-train_ind, ]

head(all_falls_train)
dim(all_falls_train)

head(all_falls_test)
dim(all_falls_test)

,X,id,recclass,mass,fell_or_found,year,latitude,longitude,group,chondrite,type,lvl3,lvl4,lc_sample,weights
,<int>,<int>,<chr>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
503,4558,12465,L4,7000,Fell,1925,47.75000,16.23333,L,Chondrite,Stony,Ordinary,L,2,0.06434506
985,6750,24143,H6,40,Fell,1668,45.41667,11.13333,H,Chondrite,Stony,Ordinary,H,1,0.02949758
1004,6801,24189,H6,594,Fell,1915,25.45000,70.00000,H,Chondrite,Stony,Ordinary,H,14,0.02860885
919,6444,23760,H6,7235,Fell,1865,26.71667,84.21667,H,Chondrite,Stony,Ordinary,H,1,0.02949758
470,4443,12342,L6,11510,Fell,1986,34.30000,133.95000,L,Chondrite,Stony,Ordinary,L,11,0.00000000
823,5821,22792,H5,1250,Fell,1855,27.43333,82.08333,H,Chondrite,Stony,Ordinary,H,1,0.02949758


[1] 784  15

,X,id,recclass,mass,fell_or_found,year,latitude,longitude,group,chondrite,type,lvl3,lvl4,lc_sample,weights
,<int>,<int>,<chr>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
3,5,6,EH4,107000,Fell,1952,54.21667,-113.00000,EH,Chondrite,Stony,Enstatite,EH-EL,1,0.02949758
5,367,370,L6,780,Fell,1902,-33.16667,-64.95000,L,Chondrite,Stony,Ordinary,L,2,0.06434506
7,385,390,LL3-6,910,Fell,1949,44.83333,95.16667,LL,Chondrite,Stony,Ordinary,LL,2,0.06434506
8,386,392,H5,30000,Fell,1814,44.21667,0.61667,H,Chondrite,Stony,Ordinary,H,2,0.06434506
20,438,448,CI1,6000,Fell,1806,44.11667,4.08333,CI,Chondrite,Stony,Carbonaceous,CI,11,0.00000000
25,451,462,L6,3200,Fell,1873,36.23333,37.13333,L,Chondrite,Stony,Ordinary,L,2,0.06434506


[1] 336  15

In [4]:
# all

set.seed(100)
sample_size <- floor(0.7 * nrow(all))

train_ind <- sample(seq_len(nrow(all)), size = sample_size)

all_train <- all[train_ind, ]
all_test <- all[-train_ind, ]

head(all_train)
dim(all_train)

head(all_test)
dim(all_test)

,X,id,recclass,mass,fell_or_found,year,latitude,longitude,group,chondrite,type,lvl3,lvl4,lc_sample,weights
,<int>,<int>,<chr>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
3786,3786,11632,L4,633.00,Found,1995,28.69167,13.21967,L,Chondrite,Stony,Ordinary,L,11,0.0000000000
503,503,2320,L6,3200.00,Fell,1803,43.86667,5.38333,L,Chondrite,Stony,Ordinary,L,12,0.0003157322
3430,3430,10154,H4,0.83,Found,1992,-30.73783,127.95367,H,Chondrite,Stony,Ordinary,H,11,0.0000000000
3696,3696,11542,Eucrite-mmict,143.00,Found,1994,28.94550,13.06950,Eucrite,Achondrite,Stony,Asteroidal,H-E-D Vesta,11,0.0000000000
4090,4090,11944,L6,23.60,Found,1991,-30.27467,129.01850,L,Chondrite,Stony,Ordinary,L,11,0.0000000000
7886,7886,45918,LL6,425.00,Found,2006,20.01352,56.40697,LL,Chondrite,Stony,Ordinary,LL,11,0.0000000000


[1] 6927   15

,X,id,recclass,mass,fell_or_found,year,latitude,longitude,group,chondrite,type,lvl3,lvl4,lc_sample,weights
,<int>,<int>,<chr>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
3,3,4,H5,331,Found,1982,26.80000,-105.41667,H,Chondrite,Stony,Ordinary,H,2,0.06434506
8,8,9,H4,4500,Found,1997,27.23944,29.83583,H,Chondrite,Stony,Ordinary,H,11,0.00000000
11,11,12,H5,228,Found,1989,27.61667,3.85000,H,Chondrite,Stony,Ordinary,H,14,0.02860885
12,12,13,H5,145,Found,1989,27.81667,4.03333,H,Chondrite,Stony,Ordinary,H,14,0.02860885
15,15,16,H3.9/4,561,Found,1989,27.63333,3.96667,H,Chondrite,Stony,Ordinary,H,14,0.02860885
16,16,17,L5,542,Found,1989,27.51667,3.65000,L,Chondrite,Stony,Ordinary,L,14,0.02860885


[1] 2970   15

In [5]:
write.csv(all_falls_train,'data/results/traintest/meteorite_falls_train.csv')

write.csv(all_falls_test,'data/results/traintest/meteorite_falls_test.csv')

write.csv(all_train,'data/results/traintest/meteorite_falls_and_found_train.csv')

write.csv(all_test,'data/results/traintest/meteorite_falls_and_found_test.csv')

In [6]:
# Applying the weights to the training sets 
# (I get an error of "too few positive probabilities" if I go above these values).

set.seed(1234)

fall_train_wt <- all_falls_train[sample(seq_len(nrow(all_falls_train)),600,prob=all_falls_train$weights),]

head(fall_train_wt)
nrow(fall_train_wt)

ff_train_wt <- all_train[sample(seq_len(nrow(all_train)),2100,prob=all_train$weights),]

head(ff_train_wt)
nrow(ff_train_wt)

,X,id,recclass,mass,fell_or_found,year,latitude,longitude,group,chondrite,type,lvl3,lvl4,lc_sample,weights
,<int>,<int>,<chr>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
359,4017,11869,H5,3500,Fell,1922,55.46667,13.78333,H,Chondrite,Stony,Ordinary,H,0,1.00000000
142,792,5169,H5,120000,Fell,1919,5.00000,48.00000,H,Chondrite,Stony,Ordinary,H,0,1.00000000
714,5370,18072,H6,515,Fell,1910,17.74333,120.45583,H,Chondrite,Stony,Ordinary,H,0,1.00000000
992,6764,24158,H5,10800,Fell,1950,56.13333,77.26667,H,Chondrite,Stony,Ordinary,H,0,1.00000000
251,3238,7757,Ureilite,300,Fell,1872,26.25000,82.00000,Ureilite,Achondrite,Stony,Primitive or asteroidal,NA,1,0.02949758
423,4340,12232,H4,950,Fell,1951,-6.83333,29.50000,H,Chondrite,Stony,Ordinary,H,0,1.00000000


[1] 600

,X,id,recclass,mass,fell_or_found,year,latitude,longitude,group,chondrite,type,lvl3,lvl4,lc_sample,weights
,<int>,<int>,<chr>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
5577,5577,22395,H6,6.1,Fell,1961,26.66667,50.15000,H,Chondrite,Stony,Ordinary,H,0,1.00000000
9701,9701,56566,"Iron, IAB-ung",732.0,Found,1979,42.31800,-90.22100,Iron IAB,Achondrite,Iron,Non-magmatic,Primative,0,1.00000000
6666,6666,24053,H,357.4,Fell,1950,58.47333,8.86667,H,Chondrite,Stony,Ordinary,H,0,1.00000000
486,486,2302,Angrite,1500.0,Fell,1869,-22.96667,-44.31667,Angrite,Achondrite,Stony,Asteroidal,NA,2,0.06434506
701,701,5071,L6,5100.0,Fell,1909,31.83333,-98.83333,L,Chondrite,Stony,Ordinary,L,2,0.06434506
6752,6752,24144,H6,2799.0,Fell,1944,11.98333,79.75000,H,Chondrite,Stony,Ordinary,H,0,1.00000000


[1] 2100

In [7]:
write.csv(fall_train_wt,'data/results/traintest/meteorite_falls_train_weighted.csv')
write.csv(ff_train_wt,'data/results/traintest/meteorite_falls_and_found_train_weighted.csv')